In [15]:
# load the required packages
using JuMP
using GLPK

## Solving your first Linear Program on Julia 

In this example, we will show how to program a simple LP using the JuMP modeling language. We focus on a simple LP  described as follows:

$$ \begin{array}{rl}
\min_{x_1,x_2}  & 3x_1 + 5x_2 \\
\text{s.t.} & x_1 \leq 4,~2x_2 \leq 12 \\
& 3x_1 + 2 x_2 \leq 18 \\
& x_1 \geq 0,~x_2 \geq 0.
\end{array} $$

Once we have obtained a mathematical formulation for an optimization problem, the first thing we have to decide on is which "solver" to use with JuMP. Notice that JuMP is only a modeling language / interface, where the heavy lifting is actually done by the "solver". Different solvers have been designed for different types of optimization problems, and their supported optimization problems can be found in the table on http://www.juliaopt.org/JuMP.jl/v0.21.1/installation/#Getting-Solvers-1

In FTEC2101/ESTR2520, where we deal with mostly LPs, IPs, and convex programs. We only focus on 
- using "GLPK" which supports LPs and IPs; 
- using "ECOS" which supports second order cone program (SOCP);
- using "Juniper" (with "Ipopt") which supports mixed-integer SOCP. 

These packages have been installed in your Julia environment with the program "install_packages_2021.ipynb". For this lab, we only need to use "GLPK" since our objective is only to solve an LP problem. 

The following code sniplet initialize an optimization problem by specifying the solver "GLPK.Optimizer" to be used:

In [16]:
model = Model(GLPK.Optimizer);

We have to specify the decision variable as follows (notice that we can include the $x_1\geq0,x_2\geq0$ constraints):

In [17]:
@variable(model, x[1:2] >= 0)

2-element Array{VariableRef,1}:
 x[1]
 x[2]

We can specify the functional constraints in matrix form. In particular, note that our constraints can be written as:

$$ \left( \begin{array}{cc} 1 & 0 \\ 0 & 2 \\ 3 & 2 \end{array} \right) \left( \begin{array}{c} x_1 \\ x_2 \end{array} \right) \leq \left( \begin{array}{c} 4 \\ 12 \\ 18 \end{array} \right) $$

In JuMP, it can be easily defined using the following code:

In [18]:
A = [1 0; 0 2; 3 2]; b = [4; 12; 18]
@constraint(model, A*x .<= b)

3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 x[1] <= 4.0            
 2 x[2] <= 12.0         
 3 x[1] + 2 x[2] <= 18.0

Notice the following rules:

1. A matrix is defined using the square bracket "[" and "]"; we can specify the values in different columns of it by adding "spaces"; and to start a new row in the matrix, we can use the semicolun ";". 
2. In Julia syntax, we use "A*x .<= b" to denote the "element-wise" comparison using the dotted "<=".
3. Notice that you can use "@constraint" for multiple times to "add" constraints to your model.

Finally, we can specify the objective function, and inspect the final optimization model:

In [19]:
@objective(model, Max, 3*x[1]+5*x[2])
print(model)

Max 3 x[1] + 5 x[2]
Subject to
 x[1] <= 4.0
 2 x[2] <= 12.0
 3 x[1] + 2 x[2] <= 18.0
 x[1] >= 0.0
 x[2] >= 0.0


In case the syntax was still unclear at this point, please note that "x[1]" (resp. "x[2]") refers to the first (resp. second) element in the vector $x$.

Solving the LP problem requires only 1 line.

In [20]:
status = optimize!(model)

Lastly, we can inspect the solution to the LP

In [21]:
print("Optimal x = ", value.(x), ".  Optimal objective value = ", objective_value(model))

Optimal x = [2.0, 6.0].  Optimal objective value = 36.0

To modify a model, note that you should re-run the jupyter notebook from the line "model = Model(solver = GLPKSolverMIP());", otherwise you may be solving a different than the one you wanted. 

## After-class Exercise

Now, let's try to write your own program solving the following LP:

$$ \begin{array}{rl}
\max_{x_1,x_2} & 2x_1 + 3x_2 \\
\text{s.t.} & x_1 + 2 x_2 \leq 4 \\
& x_1 + x_2  = 3 \\
& x_1,x_2 \geq 0.
\end{array} $$

Remark: As a good practice, for your program in the same notebook, you should use different names for the variables to avoid conflicts.

In [22]:
model2 = Model(GLPK.Optimizer);
# Your code here, you may use "y" instead of "x" for the variable here to 
# avoid "erasing" the previously stored variables 

In [23]:
@variable(model2, y[1:2]>=0)

2-element Array{VariableRef,1}:
 y[1]
 y[2]

In [24]:
A2 = [1 2; 1 1]; b2=[4; 3]
@constraint(model2, A2*y.<= b2)

2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 y[1] + 2 y[2] <= 4.0
 y[1] + y[2] <= 3.0  

In [25]:
A3= [1 1]; b3=[3]
@constraint(model2, A3*y .>=b3)

1-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 y[1] + y[2] >= 3.0

In [26]:
@objective(model2, Max, 2*y[1]+3*y[2])
print(model2)

Max 2 y[1] + 3 y[2]
Subject to
 y[1] + y[2] >= 3.0
 y[1] + 2 y[2] <= 4.0
 y[1] + y[2] <= 3.0
 y[1] >= 0.0
 y[2] >= 0.0


In [27]:
status = optimize!(model2)

In [28]:
print("Optimal y = ", value.(y), ".  Optimal objective value = ", objective_value(model2))

Optimal y = [2.0, 1.0].  Optimal objective value = 7.0